In [197]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [198]:
pickle_file = './res/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [199]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [202]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 50000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [203]:
num_steps = 8001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.673634
Training accuracy: 8.5%
Validation accuracy: 11.5%
Loss at step 100: 2.461041
Training accuracy: 71.3%
Validation accuracy: 71.0%
Loss at step 200: 2.000719
Training accuracy: 74.2%
Validation accuracy: 73.5%
Loss at step 300: 1.766876
Training accuracy: 75.3%
Validation accuracy: 74.3%
Loss at step 400: 1.615195
Training accuracy: 75.8%
Validation accuracy: 75.1%
Loss at step 500: 1.505241
Training accuracy: 76.3%
Validation accuracy: 75.5%
Loss at step 600: 1.419911
Training accuracy: 76.6%
Validation accuracy: 75.8%
Loss at step 700: 1.350521
Training accuracy: 76.9%
Validation accuracy: 75.9%
Loss at step 800: 1.292265
Training accuracy: 77.2%
Validation accuracy: 76.1%
Loss at step 900: 1.242237
Training accuracy: 77.4%
Validation accuracy: 76.3%
Loss at step 1000: 1.198561
Training accuracy: 77.6%
Validation accuracy: 76.7%
Loss at step 1100: 1.159956
Training accuracy: 77.9%
Validation accuracy: 76.8%
Loss at step 1200: 1.125503
Training acc

In [182]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  """
  定义全连接不同层的神经元个数
  """
  L = 1024
  M = 512
  N = 100
  O = 30

  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, L], stddev=0.1))
  B1 = tf.Variable(tf.ones([L])/10)

  W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
  B2 = tf.Variable(tf.ones([M])/10)

  W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
  B3 = tf.Variable(tf.ones([N])/10)
    
  W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
  B4 = tf.Variable(tf.ones([O])/10)

  W5 = tf.Variable(tf.truncated_normal([L, num_labels], stddev=0.1))
  B5 = tf.Variable(tf.zeros([num_labels]))

    
  """
  用relu作为激活函数，然后传递到下一层，最后一层的数量是输出分类的数量。
  """  
  XX_Train = tf.reshape(tf_train_dataset,[-1, image_size * image_size])
  Y1 = tf.nn.relu(tf.matmul(XX_Train, W1) + B1)
  Y2 = tf.nn.relu(tf.matmul(Y1, W2) + B2)    
  Y3 = tf.nn.relu(tf.matmul(Y2, W3) + B3)    
  Y4 = tf.nn.relu(tf.matmul(Y3, W4) + B4)    
  logits = tf.matmul(Y1, W5) + B5

  XX_valid = tf.reshape(tf_valid_dataset,[-1, image_size * image_size])
  YV1 = tf.nn.relu(tf.matmul(XX_valid, W1) + B1)
  YV2 = tf.nn.relu(tf.matmul(YV1, W2) + B2)    
  YV3 = tf.nn.relu(tf.matmul(YV2, W3) + B3)    
  YV4 = tf.nn.relu(tf.matmul(YV3, W4) + B4)    
  logits_valid = tf.matmul(YV1, W5) + B5

  XX_test = tf.reshape(tf_test_dataset,[-1, image_size * image_size])
  YT1 = tf.nn.relu(tf.matmul(XX_test, W1) + B1)
  YT2 = tf.nn.relu(tf.matmul(YT1, W2) + B2)    
  YT3 = tf.nn.relu(tf.matmul(YT2, W3) + B3)    
  YT4 = tf.nn.relu(tf.matmul(YT3, W4) + B4)    
  logits_test = tf.matmul(YT1, W5) + B5    
    
    
  # Training computation.
  
#   logits = tf.matmul(tf_train_dataset, weights) + biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_valid)
  test_prediction = tf.nn.softmax(logits_test)

#   train_prediction = tf.nn.softmax(logits)
#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf_valid_dataset, weights) + biases)
#   test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [195]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.112168
Minibatch accuracy: 5.5%
Validation accuracy: 33.0%
Minibatch loss at step 1000: 0.384361
Minibatch accuracy: 90.6%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.356991
Minibatch accuracy: 90.6%
Validation accuracy: 87.4%
Minibatch loss at step 3000: 0.375073
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 4000: 0.423447
Minibatch accuracy: 89.1%
Validation accuracy: 88.2%
Minibatch loss at step 5000: 0.205476
Minibatch accuracy: 93.0%
Validation accuracy: 88.1%
Minibatch loss at step 6000: 0.366519
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 7000: 0.371722
Minibatch accuracy: 91.4%
Validation accuracy: 87.5%
Minibatch loss at step 8000: 0.171380
Minibatch accuracy: 94.5%
Validation accuracy: 87.6%
Test accuracy: 93.3%
